# クイックスタート: Azure AI Search でのエージェント検索

このノートブックを使用して、Azure AI Search の[エージェント検索](https://learn.microsoft.com/azure/search/search-agentic-retrieval-concept)を開始しましょう。エージェント検索は会話履歴と Azure OpenAI の大規模言語モデル（LLM）を統合して、複雑なクエリの計画、検索、合成を行います。

このノートブックの手順:

+ `earth_at_night` 検索インデックスの作成

+ GitHub URL からのドキュメントをインデックスに読み込み

+ クエリ計画のためのLLMを指す `earth-search-agent` をAzure AI Searchで作成

+ エージェントを使用してインデックスから関連情報を取得しランキング

+ Azure OpenAI クライアントを使用した回答生成

このノートブックはエージェント検索の高レベルなデモを提供します。詳細なガイダンスについては、[クイックスタート: Azure AI Search でエージェント検索を実行する](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval)を参照してください。

## 前提条件

+ [セマンティックランカーが有効](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable)になっている Basic 以上のティアの [Azure AI Search サービス](https://learn.microsoft.com/azure/search/search-create-service-portal)

+ [Azure OpenAI リソース](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource)

+ Azure OpenAI リソースにデプロイされた[サポートされているモデル](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models)。このノートブックでは `gpt-4.1-mini` を使用します。

## アクセス設定

このノートブックは Microsoft Entra ID とロール割り当てを使用した認証と認可を前提としています。また、ローカルデバイスからコードを実行することも想定しています。

ロールベースアクセスを設定するには:

1. [Azure ポータル](https://portal.azure.com)にサインインします。

1. Azure AI Search サービスで[ロールベースアクセスを有効](https://learn.microsoft.com/azure/search/search-security-enable-roles)にします。

1. Azure AI Search サービスで[システム割り当てマネージド ID を作成](https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources#create-a-system-managed-identity)します。

1. Azure AI Search サービスで、自分に[次のロールを割り当て](https://learn.microsoft.com/azure/search/search-security-rbac#how-to-assign-roles-in-the-azure-portal)ます。

   + **Search Service Contributor**

   + **Search Index Data Contributor**

   + **Search Index Data Reader**

1. Azure OpenAI リソースで、検索サービスのマネージド ID に **Cognitive Services User** を割り当てます。

## 接続設定

`sample.env` ファイルには、Azure AI Search と Azure OpenAI への接続用の環境変数が含まれています。エージェント検索には、ドキュメント検索、クエリ計画、クエリ実行、回答生成のためにこれらの接続が必要です。

接続を設定するには:

1. [Azure ポータル](https://portal.azure.com)にサインインします。

2. Azure AI Search と Azure OpenAI の両方のエンドポイントを取得します。

3. `sample.env` ファイルをローカルデバイスに `.env` として保存します。

4. 取得したエンドポイントで `.env` ファイルを更新します。

## 仮想環境の作成

`requirements.txt` ファイルには、このノートブックの依存関係が含まれています。仮想環境を使用して、これらの依存関係を分離してインストールできます。

仮想環境を作成するには:

1. Visual Studio Code で、`quickstart.ipynb` を含むフォルダを開きます。

1. **Ctrl**+**Shift**+**P** を押してコマンドパレットを開きます。

1. **Python: Create Environment** を検索し、**Venv** を選択します。

1. Python インストールを選択します。このノートブックは Python 3.13 でテストしました。

1. 依存関係として `requirements.txt` を選択します。

仮想環境の作成には数分かかる場合があります。環境が準備できたら、次のステップに進みます。

## パッケージのインストールと接続の読み込み

このステップでは、このノートブック用のパッケージをインストールし、Azure AI Search と Azure OpenAI への接続を確立します。

In [ ]:
!az login

In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # Take environment variables from .env.

# The following variables from your .env file are used in this notebook
answer_model = os.getenv("ANSWER_MODEL", "gpt-4.1-mini")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night_4")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-03-01-preview")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent-4")
api_version = "2025-05-01-Preview"

## Azure AI Search でのインデックス作成

このステップでは、プレーンテキストとベクターコンテンツを含む検索インデックスを作成します。既存のインデックスを使用することもできますが、[エージェント検索ワークロード](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index)の基準を満たす必要があります。主要なスキーマ要件は、`default_configuration_name` を持つセマンティック設定です。

In [ ]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")

## サンプルドキュメントのアップロード

このノートブックは NASA の「Earth at Night」電子書籍のデータを使用します。データは GitHub の [azure-search-sample-data](https://github.com/Azure-Samples/azure-search-sample-data) リポジトリから取得され、インデックス化のために検索クライアントに渡されます。

In [ ]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}'")

## Azure AI Search でのエージェント作成

このステップでは、Azure OpenAI にデプロイした LLM のラッパーとして機能するナレッジエージェントを作成します。LLM はエージェント検索パイプラインにクエリを送信するために使用されます。

In [ ]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex, KnowledgeAgentRequestLimits, AzureOpenAIVectorizerParameters

agent = KnowledgeAgent(
    name=agent_name,
    models=[
        KnowledgeAgentAzureOpenAIModel(
            azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,
                deployment_name=azure_openai_gpt_deployment,
                model_name=azure_openai_gpt_model
            )
        )
    ],
    target_indexes=[
        KnowledgeAgentTargetIndex(
            index_name=index_name,
            default_reranker_threshold=2.5
        )
    ],
)

index_client.create_or_update_agent(agent)
print(f"Knowledge agent '{agent_name}' created or updated successfully")


## メッセージの設定

メッセージは検索ルートの入力で、会話履歴を含みます。各メッセージには、`assistant` や `user` など起源を示す `role` と、自然言語での `content` が含まれます。使用する LLM によって有効なロールが決まります。

In [ ]:
instructions = """
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

## エージェント検索を使用した結果の取得

このステップでは、検索インデックスから関連情報を抽出するために検索パイプラインを実行します。検索リクエストのメッセージとパラメータに基づいて、LLM は:

1. 会話履歴全体を分析して、根本的な情報ニーズを判断します。

1. 複合ユーザークエリを焦点を絞ったサブクエリに分解します。
 
1. 各サブクエリを同時に、インデックスのテキストフィールドとベクター埋め込みに対して実行します。

1. セマンティックランカーを使用して、すべてのサブクエリの結果を再ランク付けします。

1. 結果を単一の文字列にマージします。

In [ ]:
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams

agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, credential=credential)

messages.append({
    "role": "user",
    "content": """
    ダウンタウンの方が絶対的な光量が多いにもかかわらず、郊外のベルト地帯の方が都市の中心部よりも12月の輝度が高いのはなぜか？
    """
})

retrieval_result = agent_client.retrieve(
    retrieval_request=KnowledgeAgentRetrievalRequest(
        messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg["content"])]) for msg in messages if msg["role"] != "system"],
        target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2)]
    )
)
messages.append({
    "role": "assistant",
    "content": retrieval_result.response[0].content[0].text
})

### 検索レスポンス、アクティビティ、結果のレビュー

Azure AI Search からの各検索レスポンスには以下が含まれます:

+ 検索結果からのグラウンディングデータを表す統一文字列

+ クエリ計画

+ ソースドキュメントのどの部分が統一文字列に貢献したかを示す参照データ

In [ ]:
import textwrap

print("Response")
print(textwrap.fill(retrieval_result.response[0].content[0].text, width=120))

In [ ]:
import json
print("Activity")
print(json.dumps([a.as_dict() for a in retrieval_result.activity], indent=2))
print("Results")
print(json.dumps([r.as_dict() for r in retrieval_result.references], indent=2))

## Azure OpenAI クライアントの作成

ここまで、このノートブックは回答*抽出*のためにエージェント検索を使用してきました。Azure OpenAI クライアントを使用することで、これを回答*生成*まで拡張できます。これにより、インデックス化されたコンテンツに厳密に結びつかない、より詳細でコンテキストに富んだレスポンスが可能になります。

In [ ]:
from openai import AzureOpenAI
from azure.identity import get_bearer_token_provider

azure_openai_token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    azure_ad_token_provider=azure_openai_token_provider,
    api_version=azure_openai_api_version
)

### Responses API を使用した回答生成

回答生成の一つの選択肢は Responses API で、これは会話履歴を処理のために LLM に渡します。

In [ ]:
response = client.responses.create(
    model=answer_model,
    input=messages
)

wrapped = textwrap.fill(response.output_text, width=100)
print(wrapped)

In [ ]:
messages

### Chat Completions API を使用した回答生成

代替として、回答生成に Chat Completions API を使用することもできます。

In [ ]:
response = client.chat.completions.create(
    model=answer_model,
    messages=messages
)

wrapped = textwrap.fill(response.choices[0].message.content, width=100)
print(wrapped)

In [ ]:
messages

## 会話の継続

このステップでは、ナレッジエージェントとの会話を継続し、前のメッセージとクエリに基づいて、検索インデックスから関連情報を取得します。

In [ ]:
messages.append({
    "role": "user",
    "content": "How do I find lava at night?"
})

retrieval_result = agent_client.retrieve(
    retrieval_request=KnowledgeAgentRetrievalRequest(
        messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg["content"])]) for msg in messages if msg["role"] != "system"],
        target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
    )
)
messages.append({
    "role": "assistant",
    "content": retrieval_result.response[0].content[0].text
})

### 検索レスポンス、アクティビティ、結果のレビュー

In [ ]:
print("Response")
print(textwrap.fill(retrieval_result.response[0].content[0].text, width=120))

In [ ]:
import json
print("Activity")
print(json.dumps([a.as_dict() for a in retrieval_result.activity], indent=2))
print("Results")
print(json.dumps([r.as_dict() for r in retrieval_result.references], indent=2))

## 回答生成

In [ ]:
response = client.responses.create(
    model=answer_model,
    input=messages
)

wrapped = textwrap.fill(response.output_text, width=100)
print(wrapped)

## オブジェクトとリソースのクリーンアップ

Azure AI Search や Azure OpenAI が不要になった場合は、Azure サブスクリプションから削除してください。個別のオブジェクトを削除して最初からやり直すこともできます。

### ナレッジエージェントの削除

In [ ]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.delete_agent(agent_name)
print(f"Knowledge agent '{agent_name}' deleted successfully")

### 検索インデックスの削除

In [ ]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.delete_index(index)
print(f"Index '{index_name}' deleted successfully")